# AI-Powered Loan Risk Assessment with CrewAI
## ITI122 GenAI Assignment 1 - Multi-Agent Framework

**Framework:** CrewAI (Multi-Agent Orchestration) + RAG + OpenAI API

**Architecture:** Three specialized agents working together:
- 📊 Data Retrieval Agent
- 📋 Policy Research Agent  
- ⚖️ Risk Assessment Agent

## Step 1: Install Packages

In [ ]:
!pip install -q crewai crewai-tools langchain-openai langchain-community chromadb sentence-transformers
!pip install -q gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.0 MB

## Step 2: Set API Key
Get your OpenAI API key from: https://platform.openai.com/api-keys

In [ ]:
import gdown
url = 'https://drive.google.com/file/d/1f-X_cbCcJG0JrJl2FsCNceuA6POKjnXm/view?usp=drive_link'
output_path = '.env'
gdown.download(url, output_path, quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1f-X_cbCcJG0JrJl2FsCNceuA6POKjnXm
To: /content/.env
100%|██████████| 179/179 [00:00<00:00, 455kB/s]


'.env'

In [ ]:
import os
from dotenv import load_dotenv
# import getpass
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API key: ")

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
print("✅ API key configured")

✅ API key configured


## Step 3: Setup Data & RAG

In [ ]:
# Setup 3 SQLite DB to simulate the 3 legacy systems

# Download CustomerAccountStatus.db
url = "https://drive.google.com/file/d/1fknKqet48J2pfksqILoS3HHlbkkLitbJ/view?usp=drive_link"
output = "CustomerAccountStatus.db"
gdown.download(url, output, quiet=False, fuzzy=True)

# Download CustomerCreditScore.db
url = "https://drive.google.com/file/d/1pLXxEwvBJSW3XIYO0yIGNyc589o_Pthk/view?usp=drive_link"
output = "CustomerCreditScore.db"
gdown.download(url, output, quiet=False, fuzzy=True)

# Download GovernmentPRStatus.db
url = "https://drive.google.com/file/d/1qaxPbcBsOkZA6q38g0hNW-q5Ltop1ujZ/view?usp=drive_link"
output = "GovernmentPRStatus.db"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1fknKqet48J2pfksqILoS3HHlbkkLitbJ
To: /content/CustomerAccountStatus.db
100%|██████████| 8.19k/8.19k [00:00<00:00, 19.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pLXxEwvBJSW3XIYO0yIGNyc589o_Pthk
To: /content/CustomerCreditScore.db
100%|██████████| 8.19k/8.19k [00:00<00:00, 20.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qaxPbcBsOkZA6q38g0hNW-q5Ltop1ujZ
To: /content/GovernmentPRStatus.db
100%|██████████| 8.19k/8.19k [00:00<00:00, 7.51MB/s]


'GovernmentPRStatus.db'

In [ ]:
import sqlite3
# SQLite DB connection test

# Path to SQLite database file
db_path = "CustomerCreditScore.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query column names
cursor.execute("SELECT * FROM CustomerCreditScore LIMIT 1;")
print([d[0] for d in cursor.description])  # column names
print(cursor.fetchone())                  # first row

# Query all rows
print("=== All Customers ===")
cursor.execute("SELECT * FROM CustomerCreditScore;")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the DB connection
conn.close()

['ID', 'NAME', 'EMAIL', 'CreditScore']
(1111, 'Loren', 'loren@gmail.com', 455)
=== All Customers ===
(1111, 'Loren', 'loren@gmail.com', 455)
(2222, 'Matt', 'matt@yahoo.com', 685)
(3333, 'Hilda', 'halida@gmail.com', 825)
(4444, 'Andy', 'andy@gmail.com', 840)
(5555, 'Kit', 'kit@yahho.com', 350)


In [ ]:
import sqlite3
# SQLite DB connection test

# Path to SQLite database file
db_path = "GovernmentPRStatus.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query column names
cursor.execute("SELECT * FROM GovernmentPRStatus LIMIT 1;")
print([d[0] for d in cursor.description])  # column names
print(cursor.fetchone())                  # first row

# Query all rows
print("=== All Government PR Status Records ===")
cursor.execute("SELECT * FROM GovernmentPRStatus;")
for row in cursor.fetchall():
    print(row)

conn.close()

['ID', 'NAME', 'EMAIL', 'PRStatus']
(2222, 'Matt', 'matt@yahoo.com', 'true')
=== All Government PR Status Records ===
(2222, 'Matt', 'matt@yahoo.com', 'true')
(4444, 'Andy', 'andy@gmail.com', 'false')


In [ ]:
import sqlite3
# SQLite DB connection test

# Path to SQLite database file
db_path = "CustomerAccountStatus.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query column names
cursor.execute("SELECT * FROM CustomerAccountStatus LIMIT 1;")
print([d[0] for d in cursor.description])  # column names
print(cursor.fetchone())                  # first row

# Query all rows
print("=== All Customer Account Status Records ===")
cursor.execute("SELECT * FROM CustomerAccountStatus;")
for row in cursor.fetchall():
    print(row)

conn.close()

['ID', 'NAME', 'NATIONALITY', 'EMAIL', 'AccountStatus']
(1111, 'Loren', 'Singaporean', 'loren@gmail.com', 'good-standing')
=== All Customer Account Status Records ===
(1111, 'Loren', 'Singaporean', 'loren@gmail.com', 'good-standing')
(2222, 'Matt', 'Non-Singaporean', 'matt@yahoo.com', 'closed')
(3333, 'Hilda', 'Singaporean', 'halida@gmail.com', 'delinquent')
(4444, 'Andy', 'Non-Singaporean', 'andy@gmail.com', 'good-standing')
(5555, 'Kit', 'Singaporean', 'kit@yahho.com', 'delinquent')


In [ ]:
pip install spire.pdf.free


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 MB 29.8 MB/s eta 0:00:00
  Attempting uninstall: plum-dispatch
    Found existing installation: plum-dispatch 2.6.0
    Uninstalling plum-dispatch-2.6.0:
      Successfully uninstalled plum-dispatch-2.6.0


In [ ]:
# Download policies

# Download Bank Loan Overall Risk Policy
url = "https://drive.google.com/file/d/1I3ua9-ON94TLoypYsj6zmtrXzea_9M2U/view?usp=sharing"
output = "Bank Loan Overall Risk Policy.pdf"
gdown.download(url, output, quiet=False, fuzzy=True)

# Download Bank Loan Interest Rate Policy
url = "https://drive.google.com/file/d/1_0iYW7uPhKcTVmqzr8YWjcQ4rY_2Wl_V/view?usp=sharing"
output = "Bank Loan Interest Rate Policy.pdf"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1I3ua9-ON94TLoypYsj6zmtrXzea_9M2U
To: /content/Bank Loan Overall Risk Policy.pdf
100%|██████████| 35.7k/35.7k [00:00<00:00, 53.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_0iYW7uPhKcTVmqzr8YWjcQ4rY_2Wl_V
To: /content/Bank Loan Interest Rate Policy.pdf
100%|██████████| 32.4k/32.4k [00:00<00:00, 48.9MB/s]


'Bank Loan Interest Rate Policy.pdf'

In [ ]:
from spire.pdf import PdfDocument, PdfTableExtractor
import pandas as pd

# Load PDF document
pdf = PdfDocument()
pdf.LoadFromFile("Bank Loan Overall Risk Policy.pdf")

# Create a PdfTableExtractor object
table_extractor = PdfTableExtractor(pdf)

dfs = []

# Loop pages
for i in range(pdf.Pages.Count):
    tables = table_extractor.ExtractTable(i)

    for table_index, table in enumerate(tables):
        print(f"Table {table_index + 1} on page {i + 1}:")

        rows = []  # store all rows here

        # Loop rows
        for r in range(table.GetRowCount()):
            row_data = []

            # Loop columns
            for c in range(table.GetColumnCount()):
                text = table.GetText(r, c)
                text = text.replace("\n", " ").strip()
                row_data.append(text)

            rows.append(row_data)

        # Convert rows → DataFrame
        df = pd.DataFrame(rows)
        dfs.append(df)

bank_loan_overall_risk_df = pd.concat(dfs, ignore_index=True)
bank_loan_overall_risk_df

Table 1 on page 1:


,0,1,2
0,Credit Score,Account Status,Overall Risk
1,300 – 674,Delinquent,High
2,675 - 749,Delinquent,High
3,750 - 850,Delinquent,Medium
4,300 – 674,Closed,High
5,675 - 749,Closed,Medium
6,750 - 850,Closed,Low
7,300 – 674,Good-standing,Medium
8,675 - 749,Good-standing,Medium
9,750 - 850,Good-standing,Low


In [ ]:
# Load PDF document
pdf = PdfDocument()
pdf.LoadFromFile("Bank Loan Interest Rate Policy.pdf")

# Create a PdfTableExtractor object
table_extractor = PdfTableExtractor(pdf)

dfs = []

# Loop pages
for i in range(pdf.Pages.Count):
    tables = table_extractor.ExtractTable(i)

    for table_index, table in enumerate(tables):
        print(f"Table {table_index + 1} on page {i + 1}:")

        rows = []  # store all rows here

        # Loop rows
        for r in range(table.GetRowCount()):
            row_data = []

            # Loop columns
            for c in range(table.GetColumnCount()):
                text = table.GetText(r, c)
                text = text.replace("\n", " ").strip()
                row_data.append(text)

            rows.append(row_data)

        # Convert rows → DataFrame
        df = pd.DataFrame(rows)
        dfs.append(df)

bank_loan_interest_rate_df = pd.concat(dfs, ignore_index=True)
bank_loan_interest_rate_df

Table 1 on page 1:


,0,1
0,Overall Risk,Interest Rate
1,Low,3.175 %
2,Medium,4.885 %
3,High,6.325 %


In [ ]:
# Merge both policies dataframes into single dataframe

# 1) Set column names
bank_loan_overall_risk_df.columns = ["Credit Score", "Account Status", "Overall Risk"]
bank_loan_interest_rate_df.columns = ["Overall Risk", "Interest Rate"]

# 2) Remove the header rows that were captured as data
# For overall risk table
bank_loan_overall_risk_df = bank_loan_overall_risk_df[
    bank_loan_overall_risk_df["Credit Score"].str.strip().str.lower() != "credit score"
].copy()

# For interest rate table
bank_loan_interest_rate_df = bank_loan_interest_rate_df[
    bank_loan_interest_rate_df["Overall Risk"].str.strip().str.lower() != "overall risk"
].copy()

# 3) Normalize whitespace
for col in ["Credit Score", "Account Status", "Overall Risk"]:
    bank_loan_overall_risk_df[col] = bank_loan_overall_risk_df[col].astype(str).str.strip()

for col in ["Overall Risk", "Interest Rate"]:
    bank_loan_interest_rate_df[col] = bank_loan_interest_rate_df[col].astype(str).str.strip()

# 4) Merge
bank_loan_policies_df = bank_loan_overall_risk_df.merge(
    bank_loan_interest_rate_df,
    on="Overall Risk",
    how="left"
).reset_index(drop=True)

bank_loan_policies_df

,Credit Score,Account Status,Overall Risk,Interest Rate
0,300 – 674,Delinquent,High,6.325 %
1,675 - 749,Delinquent,High,6.325 %
2,750 - 850,Delinquent,Medium,4.885 %
3,300 – 674,Closed,High,6.325 %
4,675 - 749,Closed,Medium,4.885 %
5,750 - 850,Closed,Low,3.175 %
6,300 – 674,Good-standing,Medium,4.885 %
7,675 - 749,Good-standing,Medium,4.885 %
8,750 - 850,Good-standing,Low,3.175 %


In [ ]:
from langchain_core.documents import Document

structured_docs = []
for _, row in bank_loan_policies_df.iterrows():
    content = (
        f"Credit Score: {row['Credit Score']}\n"
        f"Account Status: {row['Account Status']}\n"
        f"Overall Risk: {row['Overall Risk']}\n"
        f"Interest Rate: {row['Interest Rate']}\n"
    )

    structured_docs.append(
        Document(
            page_content=content,
            metadata={
                "source": "Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy",
                "credit_score_band": row["Credit Score"],
                "account_status": row["Account Status"],
                "overall_risk": row["Overall Risk"],
                "interest_rate": row["Interest Rate"],
            }
        )
    )

In [ ]:
import shutil
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Delete the existing Chroma vector database on disk
# This is to future proof as policies are reviewed regularly and this will avoid mixing old policies data
shutil.rmtree("./creditpolicy_db", ignore_errors=True)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)

vectorstore = Chroma.from_documents(
    documents=structured_docs,
    embedding=embeddings,
    collection_name="creditpolicy",
    persist_directory="./creditpolicy_db"
)

In [ ]:
# Test data retrieval
docs_with_scores = vectorstore.similarity_search_with_score("interest rate for medium risk", k=3)

for i, (doc, score) in enumerate(docs_with_scores):
    print(f"\nResult #{i+1}")
    print("Similarity score:", score)
    print("Metadata:", doc.metadata)
    print("Content:")
    print(doc.page_content)
    print("-" * 50)


Result #1
Similarity score: 0.8568880558013916
Metadata: {'interest_rate': '4.885 %', 'credit_score_band': '300 – 674', 'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'overall_risk': 'Medium', 'account_status': 'Good-standing'}
Content:
Credit Score: 300 – 674
Account Status: Good-standing
Overall Risk: Medium
Interest Rate: 4.885 %

--------------------------------------------------

Result #2
Similarity score: 0.8894208669662476
Metadata: {'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'interest_rate': '4.885 %', 'overall_risk': 'Medium', 'account_status': 'Good-standing', 'credit_score_band': '675 - 749'}
Content:
Credit Score: 675 - 749
Account Status: Good-standing
Overall Risk: Medium
Interest Rate: 4.885 %

--------------------------------------------------

Result #3
Similarity score: 0.900327205657959
Metadata: {'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'account_status': 'Delinqu

## Step 4: Create Custom Tools

In [ ]:
from crewai.tools import BaseTool # NOT crewai_tools
from typing import Type, Optional, Dict, Any
from pydantic import BaseModel, Field
import json

SQLITE_PATH = "./customer_records.db"

# Helper function
def _norm(s: str) -> str:
    return s.strip().lower()

def find_customer_row(conn, table: str, identifier: str):
    ident = identifier.strip().lower()
    cur = conn.cursor()

    cur.execute(f"""
        SELECT *
        FROM {table}
        WHERE CAST(ID AS TEXT) = ?
           OR lower(NAME) = ?
           OR lower(EMAIL) = ?
        LIMIT 1
    """, (ident, ident, ident))

    row = cur.fetchone()
    if not row:
        return None

    cols = [d[0] for d in cur.description]
    return dict(zip(cols, row))

# Tool Input Schemas
class CustomerInput(BaseModel):
    customer_identifier: str = Field(..., description="Customer ID, name, or email")

class PolicyQueryInput(BaseModel):
    query: str = Field(..., description="Natural language query about bank loan policy rules")
    k: int = Field(3, description="Number of candidates to retrieve")

# Credit Score Tool
CREDIT_DB_PATH = "./CustomerCreditScore.db"

class CreditScoreTool(BaseTool):
    name: str = "Credit Score Lookup"
    description: str = "Retrieves customer credit score from CustomerCreditScore.db (CustomerCreditScore table)."
    args_schema: type[BaseModel] = CustomerInput

    def _run(self, customer_identifier: str) -> str:
        conn = sqlite3.connect(CREDIT_DB_PATH)

        row = find_customer_row(conn, "CustomerCreditScore", customer_identifier)
        conn.close()

        if not row:
            return json.dumps(
                {"error": "Customer not found in CustomerCreditScore", "customer_identifier": customer_identifier},
                indent=2
            )

        return json.dumps({
            "id": row["ID"],
            "name": row["NAME"],
            "email": row["EMAIL"],
            "credit_score": row["CreditScore"]   # <-- fixed column name
        }, indent=2)

# Account Status Tool
ACCOUNT_DB_PATH = "./CustomerAccountStatus.db"

class AccountStatusTool(BaseTool):
    name: str = "Account Status Lookup"
    description: str = "Retrieves customer account status and nationality from CustomerAccountStatus.db (CustomerAccountStatus table)."
    args_schema: type[BaseModel] = CustomerInput

    def _run(self, customer_identifier: str) -> str:
        conn = sqlite3.connect(ACCOUNT_DB_PATH)

        row = find_customer_row(conn, "CustomerAccountStatus", customer_identifier)
        conn.close()

        if not row:
            return json.dumps({"error": "Customer not found in CustomerAccountStatus", "customer_identifier": customer_identifier}, indent=2)

        return json.dumps({
            "id": row["ID"],
            "name": row["NAME"],
            "email": row["EMAIL"],
            "nationality": row["NATIONALITY"],
            "account_status": row["AccountStatus"]
        }, indent=2)

# PR Status Tool
GOV_DB_PATH = "./GovernmentPRStatus.db"

class PRStatusTool(BaseTool):
    name: str = "PR Status Check"
    description: str = "Checks Permanent Resident (PR) status from GovernmentPRStatus.db (GovernmentPRStatus table)."
    args_schema: type[BaseModel] = CustomerInput

    def _run(self, customer_identifier: str) -> str:
        conn = sqlite3.connect(GOV_DB_PATH)

        row = find_customer_row(conn, "GovernmentPRStatus", customer_identifier)
        conn.close()

        if not row:
            return json.dumps({
                "id": None,
                "pr_status": None,
                "note": "No PR record found. Customer may be Singaporean or PR record not available."
            }, indent=2)

        # Your DB stores PRSTATUS as 'true'/'false' strings
        pr_raw = str(row["PRStatus"]).strip().lower()
        pr_bool = True if pr_raw == "true" else False if pr_raw == "false" else None

        return json.dumps({
            "id": row["ID"],
            "name": row["NAME"],
            "email": row["EMAIL"],
            "pr_status": pr_bool,
            "pr_status_raw": row["PRStatus"]
        }, indent=2)

# RAG Policy Search Tool
class PolicySearchTool(BaseTool):
    name: str = "Policy Document Search"
    description: str = "Retrieves structured policy rules (row-level) from Chroma vector DB built from merged policy dataframe."
    args_schema: type[BaseModel] = PolicyQueryInput

    def _run(self, query: str, k: int = 3) -> str:
        results = vectorstore.similarity_search_with_score(query, k=k)

        payload = []
        for doc, score in results:
            payload.append({
                "score": float(score),
                "metadata": doc.metadata,
                "content": doc.page_content
            })

        return json.dumps({
            "query": query,
            "top_k": payload
        }, indent=2)

# Initialize tools
credit_tool = CreditScoreTool()
account_tool = AccountStatusTool()
pr_tool = PRStatusTool()
policy_tool = PolicySearchTool()

print("✅ Custom tools created")

✅ Custom tools created


In [ ]:
# Quick test on custom tools
credit_tool = CreditScoreTool()
acct_tool = AccountStatusTool()
pr_tool = PRStatusTool()

print(credit_tool.run(customer_identifier="1111"))
print(acct_tool.run(customer_identifier="Loren"))
print(pr_tool.run(customer_identifier="Matt"))

Using Tool: Credit Score Lookup
{
  "id": 1111,
  "name": "Loren",
  "email": "loren@gmail.com",
  "credit_score": 455
}
Using Tool: Account Status Lookup
{
  "id": 1111,
  "name": "Loren",
  "email": "loren@gmail.com",
  "nationality": "Singaporean",
  "account_status": "good-standing"
}
Using Tool: PR Status Check
{
  "id": 2222,
  "name": "Matt",
  "email": "matt@yahoo.com",
  "pr_status": true,
  "pr_status_raw": "true"
}


## Step 5: Create CrewAI Agents

In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Initialize LLM - Using GPT-4o (optimal balance of performance and cost)
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    api_key=os.environ["OPENAI_API_KEY"]
)

# Agent 1: Data Retrieval Specialist
data_agent = Agent(
    role="Data Retrieval Specialist",
    goal="Gather complete customer information from all banking systems",
    backstory="""You are an expert at navigating multiple banking systems to retrieve customer data. You efficiently collect credit scores, account status, nationality, and PR status (for Non-Singaporeans) from separate databases. You ensure all required information is obtained before passing it to the assessment team.""",
    tools=[credit_tool, account_tool, pr_tool],
    llm=llm,
    verbose=True
)

# Agent 2: Policy Research Analyst
policy_agent = Agent(
    role="Policy Research Analyst",
    goal="Extract relevant loan policies and rules from bank documents",
    # backstory="""You are a meticulous policy researcher who specializes in finding and interpreting bank loan policies. You use semantic search to locate relevant risk assessment criteria and interest rate tables from policy documents. You have deep knowledge of compliance requirements, especially regarding Non-Singaporean applicants and PR status requirements.""",
    backstory="""You are a meticulous policy researcher who specializes in finding and interpreting bank loan policies. You use semantic search to locate relevant risk assessment criteria and interest rate tables from policy documents.""",
    tools=[policy_tool],
    llm=llm,
    verbose=True
)

# Agent 3: Risk Assessment Officer
risk_agent = Agent(
    role="Senior Risk Assessment Officer",
    goal="Provide accurate loan risk assessment and final recommendation",
    backstory="""You are a senior risk assessment officer with years of experience in loan evaluation. You synthesize customer data and policy information to determine overall risk levels and applicable interest rates. You strictly enforce compliance rules, especially the mandatory rejection of Non-Singaporean applicants without PR status. You provide clear, well-reasoned recommendations with full transparency.""",
    tools=[],  # Risk agent synthesizes info from other agents
    llm=llm,
    verbose=True
)

print("✅ Three specialized agents created with GPT-4o")
print("   📊 Data Retrieval Specialist")
print("   📋 Policy Research Analyst")
print("   ⚖️ Senior Risk Assessment Officer")

✅ Three specialized agents created with GPT-4o
   📊 Data Retrieval Specialist
   📋 Policy Research Analyst
   ⚖️ Senior Risk Assessment Officer


## Step 6: Define Tasks

In [ ]:
def create_assessment_tasks(customer_query: str):
    """
    Creates a sequential workflow of tasks for loan risk assessment
    """

    # Task 1: Data Collection
    data_task = Task(
        description=f"""Retrieve complete customer information for: {customer_query}

        You must collect:
        1. Credit score (use Credit Score Lookup tool)
        2. Account status and nationality (use Account Status Lookup tool)
        3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)

        Provide a comprehensive summary of all retrieved data in JSON format.""",
        agent=data_agent,
        expected_output="JSON object containing customer_id, name, email, credit_score, account_status, nationality, and pr_status (if applicable)"
    )

    # Task 2: Policy Research
    policy_task = Task(
        description="""Retrieve policy rule(s) from the structured Chroma vector DB (merged dataframe) to find:

        1. Overall risk level determination rules based on credit score and account status
        2. Interest rate table based on risk levels

        Use the Policy Document Search tool with relevant queries.
        Extract and summarize the applicable policies clearly.""",
        agent=policy_agent,
        expected_output="Clear summary of risk assessment criteria, interest rate table, and compliance rules (if applicable) from policy documents"
    )

    # Task 3: Risk Assessment & Final Recommendation
    assessment_task = Task(
        description="""Based on the customer data and policy information gathered by your team:

        1. Determine the overall risk level (Low/Medium/High) using the policy matrix
        2. Identify the applicable interest rate for that risk level
        3. Apply the mandatory compliance rule: Non-Singaporeans without PR status MUST be REJECTED
        4. Provide final recommendation: APPROVE or REJECT

        STRICT RULES:
        - Use ONLY facts from data_task output (SQLite) for customer attributes (credit_score, account_status, nationality, pr_status).
        - Use ONLY policy_task output (Chroma structured rows) as policy evidence.
        - Do NOT invent values. If required fields are missing, return decision="ERROR" and include reason.

        Your output must include:
        - Customer Information Summary
        - Overall Risk Level with justification
        - Applicable Interest Rate
        - Final Recommendation (APPROVE/REJECT)
        - Detailed Rationale explaining the decision

        Format your response as a professional loan assessment report.""",
        agent=risk_agent,
        expected_output="Comprehensive loan risk assessment report with risk level, interest rate, recommendation, and detailed rationale",
        context=[data_task, policy_task]  # This task depends on previous two tasks
    )

    return [data_task, policy_task, assessment_task]

print("✅ Task workflow defined")

✅ Task workflow defined


## Step 7: Run Test Cases

In [ ]:
def assess_loan_application(customer_query: str):
    """Run loan assessment with CrewAI multi-agent system"""
    print(f"\n{'='*80}")
    print(f"LOAN APPLICATION ASSESSMENT")
    print(f"Query: {customer_query}")
    print(f"{'='*80}\n")

    # Create tasks
    tasks = create_assessment_tasks(customer_query)

    # Create crew
    crew = Crew(
        agents=[data_agent, policy_agent, risk_agent],
        tasks=tasks,
        process=Process.sequential,  # Tasks run one after another
        verbose=True
    )

    # Execute
    result = crew.kickoff()

    print(f"\n{'='*80}")
    print("FINAL ASSESSMENT REPORT")
    print(f"{'='*80}\n")
    print(result)

    return result

print("✅ Assessment function ready")

✅ Assessment function ready


In [ ]:
results = vectorstore.similarity_search_with_score(
    query="customer with credit score 720 and closed account",
    k=3
)

results

[(Document(metadata={'overall_risk': 'Low', 'credit_score_band': '750 - 850', 'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'account_status': 'Closed', 'interest_rate': '3.175 %'}, page_content='Credit Score: 750 - 850\nAccount Status: Closed\nOverall Risk: Low\nInterest Rate: 3.175 %\n'),
  0.7746990919113159),
 (Document(metadata={'interest_rate': '4.885 %', 'credit_score_band': '675 - 749', 'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'account_status': 'Closed', 'overall_risk': 'Medium'}, page_content='Credit Score: 675 - 749\nAccount Status: Closed\nOverall Risk: Medium\nInterest Rate: 4.885 %\n'),
  0.7820608019828796),
 (Document(metadata={'overall_risk': 'High', 'interest_rate': '6.325 %', 'source': 'Bank Loan Overall Risk Policy and Bank Loan Interest Rate Policy', 'account_status': 'Closed', 'credit_score_band': '300 – 674'}, page_content='Credit Score: 300 – 674\nAccount Status: Closed\nOverall Risk: High\nInter

### Test Case 1: Loren (Expected: Medium Risk, 4.885%, APPROVE)

In [ ]:
result1 = assess_loan_application("customer ID 1111")


LOAN APPLICATION ASSESSMENT
Query: customer ID 1111



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8088d000-7c75-4b6b-b5b1-0d359bdd4932                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve complete customer information for: customer ID 1111                                             │
│                                                                                                                 │
│          You must collect:                                                                                      │
│          1. Credit score (use Credit Score Lookup tool)                                                         │
│          2. Account status and nationality (use Account Status Lookup tool)                                     │
│          3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)                          │
│                                                                                                                 │
│          Provide a comprehensive summary of all retrieved data in JSON format.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6f843ae7-63c7-498c-8406-5a3d13b03f54                                                                     │
│  Agent: Policy Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Task: Based on the customer data and policy information gathered by your team:                                 │
│                                                                                                                 │
│          1. Determine the overall risk level (Low/Medium/High) using the policy matrix                          │
│          2. Identify the applicable interest rate for that risk level                                           │
│          3. Apply the mandatory compliance rule: Non-Singaporeans without PR status MUST be REJECTED            │
│          4. Provide final recommendation: APPROVE or REJECT                                                     │
│                                                                                                                 │
│          STRICT RULES:                                                                                          │
│          - Use ONLY facts from data_task output (SQLite) for customer attributes (credit_score,                 │
│  account_status, nationality, pr_status).                                                                       │
│          - Use ONLY policy_task output (Chroma structured rows) as policy evidence.                             │
│          - Do NOT invent values. If required fields are missing, return decision="ERROR" and include reason.    │
│                                                                                                                 │
│          Your output must include:                                                                              │
│          - Customer Information Summary                                                                         │
│          - Overall Risk Level with justification                                                                │
│          - Applicable Interest Rate                                                                             │
│          - Final Recommendation (APPROVE/REJECT)                                                                │
│          - Detailed Rationale explaining the decision                                                           │
│                                                                                                                 │
│          Format your response as a professional loan assessment report.                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - Customer ID: 1111                                                                                            │
│  - Name: Loren                                                                                                  │
│  - Email: loren@gmail.com                                                                                       │
│  - Credit Score: 455                                                                                            │
│  - Account Status: Good-standing                                                                                │
│  - Nationality: Singaporean                                                                                     │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: The customer's credit score is 455, which falls within the range of 300-674. Coupled with the   │
│  account status being in good-standing, the overall risk level is determined as medium according to our policy  │
│  matrix.                                                                                                        │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  Justification: As per our policy, for a customer with a medium risk level (credit score between 300 and 674    │
│  and account status in good-standing), the applicable interest rate is 4.885%.                                  │
│                                                                                                                 │
│  Final Recommendation: APPROVE                                                                                  │
│  Justification: The customer is a Singaporean national, thus meeting our mandatory compliance rule.             │
│  Furthermore, the customer's account is in good-standing and the credit score, while not high, falls within an  │
│  acceptable range for a medium risk level. Therefore, I recommend approval of the loan.                         │
│                                                                                                                 │
│  Detailed Rationale: The decision to approve the loan is based on the customer's nationality, credit score,     │
│  and account status. The customer is a Singaporean national, which satisfies our mandatory compliance rule.     │
│  The credit score of 455, while not high, is within the acceptable range for a medium risk level. The account   │
│  status is in good-standing, which further supports the decision to approve the loan. The applicable interest   │
│  rate of 4.885% is determined based on the overall risk level.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Test Case 2: Matt (Expected: Medium Risk, 4.885%, APPROVE with PR)

In [ ]:
result2 = assess_loan_application("customer Matt")

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fa85d7f6-50b7-4d09-895c-3e42835ea4d6                                                                     │
│  Agent: Senior Risk Assessment Officer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


LOAN APPLICATION ASSESSMENT
Query: customer Matt



╭─────────────────────────────────────────── Tracing Preference Saved ────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing has been disabled.                                                                               │
│                                                                                                                 │
│  Your preference has been saved. Future Crew/Flow executions will not collect traces.                           │
│                                                                                                                 │
│  To enable tracing later, do any one of these:                                                                  │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 779e8cc9-cdc4-45de-916d-2267d40b1301                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8088d000-7c75-4b6b-b5b1-0d359bdd4932                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Customer Information Summary:                                                                    │
│  - Customer ID: 1111                                                                                            │
│  - Name: Loren                                                                                                  │
│  - Email: loren@gmail.com                                                                                       │
│  - Credit Score: 455                                                                                            │
│  - Account Status: Good-standing                                                                                │
│  - Nationality: Singaporean                                                                                     │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: The customer's credit score is 455, which falls within the range of 300-674. Coupled with the   │
│  account status being in good-standing, the overall risk level is determined as medium according to our policy  │
│  matrix.                                                                                                        │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  Justification: As per our policy, for a customer with a medium risk level (credit score between 300 and 674    │
│  and account status in good-standing), the applicable interest rate is 4.885%.                                  │
│                                                                                                                 │
│  Final Recommendation: APPROVE                                                                                  │
│  Justification: The customer is a Singaporean national, thus meeting our mandatory compliance rule.             │
│  Furthermore, the customer's account is in good-standing and the credit score, while not high, falls within an  │
│  acceptable range for a medium risk level. Therefore, I recommend approval of the loan.                         │
│                                                                                                                 │
│  Detailed Rationale: The decision to approve the loan is based on the customer's nationality, credit score,     │
│  and account status. The customer is a Singaporean national, which satisfies our mandatory compliance rule.     │
│  The credit score of 455, while not high, is within the acceptable range for a medium risk level. The account   │
│  status is in good-standing, which further supports the decision to approve the loan. The applicable interest   │
│  rate of 4.885% is determined based on the overall risk level.                                                  │
│                                                                                                                 │
│                                                       

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 28f1992b-0c59-4837-a3a6-68140dfcb03e                                                                     │
│  Agent: Policy Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - Customer ID: 2222                                                                                            │
│  - Name: Matt                                                                                                   │
│  - Email: matt@yahoo.com                                                                                        │
│  - Credit Score: 685                                                                                            │
│  - Account Status: Closed                                                                                       │
│  - Nationality: Non-Singaporean                                                                                 │
│  - PR Status: True                                                                                              │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: According to the policy matrix, a customer with a credit score between 675 and 749 and a        │
│  closed account is classified as a medium risk.                                                                 │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  Justification: The policy matrix stipulates that a medium risk customer is subject to an interest rate of      │
│  4.885%.                                                                                                        │
│                                                                                                                 │
│  Final Recommendation: Approve                                                                                  │
│  Justification: The customer is a non-Singaporean with PR status, which complies with the mandatory rule. The   │
│  customer's credit score and account status also fall within an acceptable risk level according to the policy   │
│  matrix.                                                                                                        │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  The customer, Matt, has a credit score of 685, which falls within the medium risk category according to our    │
│  policy matrix. His account status is closed, which does not increase his risk level. Furthermore, Matt is a    │
│  non-Singaporean but holds PR status, which makes him eligible for a loan according to our mandatory            │
│  compliance rule. Therefore, considering his credit score, account status, and PR status, I recommend           │
│  approving the loan application with an interest rate of 4.885%.                                                │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

Output()


FINAL ASSESSMENT REPORT

Customer Information Summary:
- Customer ID: 2222
- Name: Matt
- Email: matt@yahoo.com
- Credit Score: 685
- Account Status: Closed
- Nationality: Non-Singaporean
- PR Status: True

Overall Risk Level: Medium
Justification: According to the policy matrix, a customer with a credit score between 675 and 749 and a closed account is classified as a medium risk.

Applicable Interest Rate: 4.885%
Justification: The policy matrix stipulates that a medium risk customer is subject to an interest rate of 4.885%.

Final Recommendation: Approve
Justification: The customer is a non-Singaporean with PR status, which complies with the mandatory rule. The customer's credit score and account status also fall within an acceptable risk level according to the policy matrix.

Detailed Rationale: 
The customer, Matt, has a credit score of 685, which falls within the medium risk category according to our policy matrix. His account status is closed, which does not increase his risk l

### Test Case 3: Andy (Expected: Low Risk BUT REJECT - no PR)

In [ ]:
result3 = assess_loan_application("customer named Andy")

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 342bdf02-bceb-46a8-a86a-9d8b85808255                                                                     │
│  Agent: Senior Risk Assessment Officer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


LOAN APPLICATION ASSESSMENT
Query: customer named Andy



╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 779e8cc9-cdc4-45de-916d-2267d40b1301                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Customer Information Summary:                                                                    │
│  - Customer ID: 2222                                                                                            │
│  - Name: Matt                                                                                                   │
│  - Email: matt@yahoo.com                                                                                        │
│  - Credit Score: 685                                                                                            │
│  - Account Status: Closed                                                                                       │
│  - Nationality: Non-Singaporean                                                                                 │
│  - PR Status: True                                                                                              │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: According to the policy matrix, a customer with a credit score between 675 and 749 and a        │
│  closed account is classified as a medium risk.                                                                 │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  Justification: The policy matrix stipulates that a medium risk customer is subject to an interest rate of      │
│  4.885%.                                                                                                        │
│                                                                                                                 │
│  Final Recommendation: Approve                                                                                  │
│  Justification: The customer is a non-Singaporean with PR status, which complies with the mandatory rule. The   │
│  customer's credit score and account status also fall within an acceptable risk level according to the policy   │
│  matrix.                                                                                                        │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  The customer, Matt, has a credit score of 685, which falls within the medium risk category according to our    │
│  policy matrix. His account status is closed, which does not increase his risk level. Furthermore, Matt is a    │
│  non-Singaporean but holds PR status, which makes him eligible for a loan according to our mandatory            │
│  compliance rule. Therefore, considering his credit score, account status, and PR status, I recommend           │
│  approving the loan application with an interest rate of 4.885%.                                                │
│                                                       

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4550afae-dfbc-4326-bf2e-2bede1d7e5cc                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Policy Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The overall risk for a customer with a credit score of 840 and account status of good-standing is low. The     │
│  interest rate for this risk level is 3.175%. The interest rate for a medium risk level is 4.885%, and for a    │
│  high risk level, it is 6.325%.                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  Customer ID: 4444                                                                                              │
│  Name: Andy                                                                                                     │
│  Email: andy@gmail.com                                                                                          │
│  Credit Score: 840                                                                                              │
│  Account Status: Good-Standing                                                                                  │
│  Nationality: Non-Singaporean                                                                                   │
│  PR Status: False                                                                                               │
│                                                                                                                 │
│  Overall Risk Level: Low                                                                                        │
│  Justification: The customer has a high credit score of 840 and their account is in good standing. According    │
│  to our policy matrix, this corresponds to a low risk level.                                                    │
│                                                                                                                 │
│  Applicable Interest Rate: 3.175%                                                                               │
│  The interest rate for a low risk level is 3.175%.                                                              │
│                                                                                                                 │
│  Final Recommendation: REJECT                                                                                   │
│  Despite the low risk level and good standing of the customer, our compliance rules mandate the rejection of    │
│  Non-Singaporean applicants without PR status.                                                                  │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  The customer, Andy, has a high credit score and is in good standing, which indicates a low risk level.         │
│  However, our company policy strictly enforces the rejection of Non-Singaporean applicants without PR status.   │
│  Despite the potential profitability of this loan, we must adhere to our compliance rules. Therefore, my final  │
│  recommendation is to reject the loan application.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()


FINAL ASSESSMENT REPORT

Customer Information Summary:
Customer ID: 4444
Name: Andy
Email: andy@gmail.com
Credit Score: 840
Account Status: Good-Standing
Nationality: Non-Singaporean
PR Status: False

Overall Risk Level: Low
Justification: The customer has a high credit score of 840 and their account is in good standing. According to our policy matrix, this corresponds to a low risk level.

Applicable Interest Rate: 3.175%
The interest rate for a low risk level is 3.175%.

Final Recommendation: REJECT
Despite the low risk level and good standing of the customer, our compliance rules mandate the rejection of Non-Singaporean applicants without PR status.

Detailed Rationale:
The customer, Andy, has a high credit score and is in good standing, which indicates a low risk level. However, our company policy strictly enforces the rejection of Non-Singaporean applicants without PR status. Despite the potential profitability of this loan, we must adhere to our compliance rules. Therefore, my f

## Step 8: Try Custom Query

In [ ]:
# Try your own query!
custom_result = assess_loan_application("Check eligibility for Hilda")


LOAN APPLICATION ASSESSMENT
Query: Check eligibility for Hilda



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e34f086c-82bc-4622-91a5-aa542f08f6b2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve complete customer information for: Check eligibility for Hilda                                  │
│                                                                                                                 │
│          You must collect:                                                                                      │
│          1. Credit score (use Credit Score Lookup tool)                                                         │
│          2. Account status and nationality (use Account Status Lookup tool)                                     │
│          3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)                          │
│                                                                                                                 │
│          Provide a comprehensive summary of all retrieved data in JSON format.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - ID: 3333                                                                                                     │
│  - Name: Hilda                                                                                                  │
│  - Email: halida@gmail.com                                                                                      │
│  - Credit Score: 825                                                                                            │
│  - Nationality: Singaporean                                                                                     │
│  - Account Status: Delinquent                                                                                   │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: Hilda's credit score of 825 is well above the average, indicating a history of responsible      │
│  credit use. However, her account status is currently delinquent, which increases her risk level. Therefore,    │
│  her overall risk level is determined as medium.                                                                │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  The bank's policy rules dictate that a medium risk level corresponds to an interest rate of 4.885%.            │
│                                                                                                                 │
│  Final Recommendation: APPROVE                                                                                  │
│  Despite Hilda's delinquent account status, her high credit score and Singaporean nationality (which complies   │
│  with the bank's mandatory rule) make her a viable candidate for loan approval.                                 │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  Hilda's high credit score indicates a history of responsible credit use, which is a positive factor in her     │
│  favor. However, her delinquent account status is a concern as it suggests potential difficulties in repaying   │
│  the loan. Nevertheless, considering her Singaporean nationality, she meets the bank's mandatory compliance     │
│  rule. Therefore, while her risk level is medium, it is recommended to approve her loan application but with a  │
│  higher interest rate of 4.885% to mitigate the risk. This decision is based on the synthesis of her customer   │
│  data and the bank's policy information.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 69fe9a58-7e48-4e3e-9146-34089d66249c                                                                     │
│  Agent: Senior Risk Assessment Officer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Step 9: Additional Dataset for Testing

In [ ]:
# Download CustomerCreditScore additional dataset
url = "https://drive.google.com/file/d/12tADalMpMoUzKzluoqhwzj2Afpa4s6jj/view?usp=drive_link"
output = "credit_scores_test.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

# Download CustomerAccountStatus additional dataset
url = "https://drive.google.com/file/d/1fanw6rAIcluBJC3cssY97DMOEAJ5HVdM/view?usp=drive_link"
output = "account_status_test.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

# Download GovernmentPRStatus additional dataset
url = "https://drive.google.com/file/d/1PvwkzL1Y7DNJG8Wu9zOyeDs1joXGaEO9/view?usp=drive_link"
output = "pr_status_test.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=12tADalMpMoUzKzluoqhwzj2Afpa4s6jj
To: /content/credit_scores_test.csv
100%|██████████| 777/777 [00:00<00:00, 2.94MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fanw6rAIcluBJC3cssY97DMOEAJ5HVdM
To: /content/account_status_test.csv
100%|██████████| 1.22k/1.22k [00:00<00:00, 4.45MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PvwkzL1Y7DNJG8Wu9zOyeDs1joXGaEO9
To: /content/pr_status_test.csv
100%|██████████| 462/462 [00:00<00:00, 1.70MB/s]


'pr_status_test.csv'

In [ ]:
import sqlite3

def show_table_schema(db_path, table):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table});")
    cols = [(r[1], r[2]) for r in cur.fetchall()]  # (name, type)
    conn.close()
    return cols

print("CustomerCreditScore schema:", show_table_schema("CustomerCreditScore.db", "CustomerCreditScore"))


CustomerCreditScore schema: [('ID', 'INTEGER'), ('NAME', 'TEXT'), ('EMAIL', 'TEXT'), ('CreditScore', 'INTEGER')]


In [ ]:
import csv

with open("credit_scores_test.csv", "r", encoding="utf-8-sig") as f:
    reader = csv.reader(f)
    headers = next(reader)
print("CSV headers:", headers)


CSV headers: ['ID', 'NAME', 'EMAIL', 'CreditScore']


In [ ]:
import sqlite3
import csv

CREDIT_CSV = "credit_scores_test.csv"
ACCOUNT_CSV = "account_status_test.csv"
PR_CSV = "pr_status_test.csv"

CREDIT_DB = "CustomerCreditScore.db"
ACCOUNT_DB = "CustomerAccountStatus.db"
PR_DB = "GovernmentPRStatus.db"


def sql_literal(value):
    """Convert Python value -> SQL literal string (hardcoded into SQL)."""
    if value is None:
        return "NULL"
    s = str(value)

    if s.strip() == "":
        return "NULL"

    # keep pure integers as integers
    try:
        if str(int(s)) == s.strip():
            return str(int(s))
    except Exception:
        pass

    s = s.replace("'", "''")
    return f"'{s}'"


def read_csv_rows(csv_path):
    with open(csv_path, "r", newline="", encoding="utf-8-sig") as f:
        reader = csv.reader(f)
        headers = next(reader)
        rows = [row for row in reader if any(cell.strip() for cell in row)]
    return headers, rows


def insert_append_hardcoded(db_path, table_name, columns, rows):
    """
    Appends rows to an existing table using ONE hardcoded INSERT statement.
    Assumes:
    - table already exists
    - CSV columns match the table columns
    """
    values_sql = []
    for row in rows:
        row = (row + [""] * len(columns))[: len(columns)]
        values_sql.append("(" + ", ".join(sql_literal(v) for v in row) + ")")

    insert_sql = f"""
    INSERT INTO {table_name} ({", ".join(columns)})
    VALUES
    {",\n    ".join(values_sql)}
    ;
    """

    conn = sqlite3.connect(db_path)
    try:
        cur = conn.cursor()
        cur.executescript(insert_sql)
        conn.commit()
    finally:
        conn.close()

    return insert_sql


# 1) Credit Score
credit_cols, credit_rows = read_csv_rows(CREDIT_CSV)
insert_append_hardcoded(CREDIT_DB, "CustomerCreditScore", credit_cols, credit_rows)
print("✅ Appended:", CREDIT_DB, "rows:", len(credit_rows))

# 2) Account Status
acct_cols, acct_rows = read_csv_rows(ACCOUNT_CSV)
insert_append_hardcoded(ACCOUNT_DB, "CustomerAccountStatus", acct_cols, acct_rows)
print("✅ Appended:", ACCOUNT_DB, "rows:", len(acct_rows))

# 3) PR Status
pr_cols, pr_rows = read_csv_rows(PR_CSV)
insert_append_hardcoded(PR_DB, "GovernmentPRStatus", pr_cols, pr_rows)
print("✅ Appended:", PR_DB, "rows:", len(pr_rows))

✅ Appended: CustomerCreditScore.db rows: 20
✅ Appended: CustomerAccountStatus.db rows: 20
✅ Appended: GovernmentPRStatus.db rows: 11


In [ ]:
# ===== Quick verification prints =====
def preview(db_path, table):
    conn = sqlite3.connect(db_path)
    try:
        cur = conn.cursor()
        cur.execute(f"PRAGMA table_info({table});")
        cols = [r[1] for r in cur.fetchall()]
        cur.execute(f"SELECT * FROM {table} LIMIT 10;")
        rows = cur.fetchall()
        print(f"\n--- {db_path} | {table} ---")
        print("Columns:", cols)
        for r in rows:
            print(r)
    finally:
        conn.close()

preview(CREDIT_DB, "CustomerCreditScore")
preview(ACCOUNT_DB, "CustomerAccountStatus")
preview(PR_DB, "GovernmentPRStatus")


--- CustomerCreditScore.db | CustomerCreditScore ---
Columns: ['ID', 'NAME', 'EMAIL', 'CreditScore']
(1111, 'Loren', 'loren@gmail.com', 455)
(2222, 'Matt', 'matt@yahoo.com', 685)
(3333, 'Hilda', 'halida@gmail.com', 825)
(4444, 'Andy', 'andy@gmail.com', 840)
(5555, 'Kit', 'kit@yahho.com', 350)
(10000, 'Taylor', 'taylor624@hotmail.com', 414)
(10001, 'Jamie', 'jamie150@outlook.com', 325)
(10002, 'Drew', 'drew523@gmail.com', 581)
(10003, 'Reese', 'reese389@outlook.com', 550)
(10004, 'Morgan', 'morgan200@gmail.com', 528)

--- CustomerAccountStatus.db | CustomerAccountStatus ---
Columns: ['ID', 'NAME', 'NATIONALITY', 'EMAIL', 'AccountStatus']
(1111, 'Loren', 'Singaporean', 'loren@gmail.com', 'good-standing')
(2222, 'Matt', 'Non-Singaporean', 'matt@yahoo.com', 'closed')
(3333, 'Hilda', 'Singaporean', 'halida@gmail.com', 'delinquent')
(4444, 'Andy', 'Non-Singaporean', 'andy@gmail.com', 'good-standing')
(5555, 'Kit', 'Singaporean', 'kit@yahho.com', 'delinquent')
(10000, 'Taylor', 'Non-Singapor

## Step 10: Additional Test Cases

In [ ]:
print("\nTest Case 4: Peyton (Expected: High Risk, 6.325%, APPROVE – PR=TRUE)")
result4 = assess_loan_application("customer Peyton")
print(result4)


Test Case 4: Peyton (Expected: High Risk, 6.325%, APPROVE – PR=TRUE)

LOAN APPLICATION ASSESSMENT
Query: customer Peyton



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1b6bc0d1-6fc4-4eb8-a6eb-afd709ce0d2c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve complete customer information for: customer Peyton                                              │
│                                                                                                                 │
│          You must collect:                                                                                      │
│          1. Credit score (use Credit Score Lookup tool)                                                         │
│          2. Account status and nationality (use Account Status Lookup tool)                                     │
│          3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)                          │
│                                                                                                                 │
│          Provide a comprehensive summary of all retrieved data in JSON format.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 23d92b44-218a-48b2-b034-9c345b9cf15f                                                                     │
│  Agent: Policy Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - Customer ID: 10007                                                                                           │
│  - Name: Peyton                                                                                                 │
│  - Email: peyton844@yahoo.com                                                                                   │
│  - Credit Score: 389                                                                                            │
│  - Account Status: Delinquent                                                                                   │
│  - Nationality: Non-Singaporean                                                                                 │
│  - PR Status: True                                                                                              │
│                                                                                                                 │
│  Overall Risk Level: High                                                                                       │
│  Justification: The customer's credit score of 389 and delinquent account status align with our policy          │
│  matrix's criteria for high risk.                                                                               │
│                                                                                                                 │
│  Applicable Interest Rate: 6.325%                                                                               │
│  Justification: Our policy matrix dictates that customers with a high overall risk level are subject to an      │
│  interest rate of 6.325%.                                                                                       │
│                                                                                                                 │
│  Final Recommendation: APPROVE                                                                                  │
│  Justification: Despite the customer's high risk level, they are not automatically disqualified from loan       │
│  consideration due to their PR status. However, the high interest rate reflects the increased risk associated   │
│  with this loan.                                                                                                │
│                                                                                                                 │
│  Detailed Rationale: The customer's credit score and account status place them in the high risk category,       │
│  which typically warrants a loan rejection. However, our policy allows for the approval of high risk loans at   │
│  a higher interest rate. The customer's PR status also allows them to be considered for a loan despite their    │
│  non-Singaporean nationality. Therefore, I recommend approving the loan with an interest rate of 6.325%.        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [ ]:
print("\nTest Case 5: Taylor (Expected: Medium Risk, 4.885%, REJECT/NOT RECOMMEND - PR false)")
result5 = assess_loan_application("customer ID 10000")
print(result5)


Test Case 5: Taylor (Expected: Medium Risk, 4.885%, REJECT/NOT RECOMMEND - PR false)

LOAN APPLICATION ASSESSMENT
Query: customer ID 10000



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b65d0b26-2dbe-44b0-8dcc-a3bb46f69f2f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve complete customer information for: customer ID 10000                                            │
│                                                                                                                 │
│          You must collect:                                                                                      │
│          1. Credit score (use Credit Score Lookup tool)                                                         │
│          2. Account status and nationality (use Account Status Lookup tool)                                     │
│          3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)                          │
│                                                                                                                 │
│          Provide a comprehensive summary of all retrieved data in JSON format.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a8d89428-d835-47ad-8f96-cc459569ee16                                                                     │
│  Agent: Policy Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - Customer ID: 10000                                                                                           │
│  - Name: Taylor                                                                                                 │
│  - Email: taylor624@hotmail.com                                                                                 │
│  - Credit Score: 414                                                                                            │
│  - Account Status: Good-standing                                                                                │
│  - Nationality: Non-Singaporean                                                                                 │
│  - PR Status: False                                                                                             │
│                                                                                                                 │
│  Overall Risk Level: Medium                                                                                     │
│  Justification: Based on our policy matrix, a customer with a credit score of 414 and an account status of      │
│  "good-standing" falls under the medium risk category.                                                          │
│                                                                                                                 │
│  Applicable Interest Rate: 4.885%                                                                               │
│  Justification: The policy matrix indicates that the interest rate for a medium risk level is 4.885%.           │
│                                                                                                                 │
│  Final Recommendation: REJECT                                                                                   │
│  Justification: Despite the customer's medium risk level and good-standing account status, our mandatory        │
│  compliance rules dictate that Non-Singaporean applicants without PR status must be rejected.                   │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  The customer's credit score and account status align with our policy for a medium risk level, which would      │
│  typically result in a loan approval with an interest rate of 4.885%. However, our mandatory compliance rules   │
│  override this assessment. As the customer is a Non-Singaporean without PR status, we are required to reject    │
│  the loan application. This decision is in strict adherence to our compliance rules and is non-negotiable.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f8259fc6-b897-418f-bdfd-b80e39362460                                                                     │
│  Agent: Senior Risk Assessment Officer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯


In [ ]:
print("\nTest Case 6: Jamie (Expected: High Risk, 6.325%, APPROVE - PR true)")
result6 = assess_loan_application("customer ID 10001")
print(result6)


Test Case 6: Jamie (Expected: High Risk, 6.325%, APPROVE - PR true)

LOAN APPLICATION ASSESSMENT
Query: customer ID 10001



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2a1ddb4c-3f0b-4595-96ee-696a8c8f8f17                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve complete customer information for: customer ID 10001                                            │
│                                                                                                                 │
│          You must collect:                                                                                      │
│          1. Credit score (use Credit Score Lookup tool)                                                         │
│          2. Account status and nationality (use Account Status Lookup tool)                                     │
│          3. If customer is Non-Singaporean, check PR status (use PR Status Check tool)                          │
│                                                                                                                 │
│          Provide a comprehensive summary of all retrieved data in JSON format.                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2268635f-17f0-4694-8477-eb07b18c4945                                                                     │
│  Agent: Policy Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Risk Assessment Officer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Customer Information Summary:                                                                                  │
│  - Customer ID: 10001                                                                                           │
│  - Name: Jamie                                                                                                  │
│  - Email: jamie150@outlook.com                                                                                  │
│  - Credit Score: 325                                                                                            │
│  - Account Status: Closed                                                                                       │
│  - Nationality: Non-Singaporean                                                                                 │
│  - PR Status: True                                                                                              │
│                                                                                                                 │
│  Overall Risk Level: High                                                                                       │
│  Justification: According to the risk level determination rules, a customer with a credit score between 300 –   │
│  674 and a closed account status is considered high risk.                                                       │
│                                                                                                                 │
│  Applicable Interest Rate: 6.325%                                                                               │
│  Justification: The interest rate for high-risk customers, as per the interest rate table, is 6.325%.           │
│                                                                                                                 │
│  Final Recommendation: APPROVE                                                                                  │
│  Justification: Although the customer is high risk, they are a PR holder. As per the mandatory compliance       │
│  rule, only Non-Singaporeans without PR status must be rejected. Since this customer has PR status, they are    │
│  eligible for loan approval.                                                                                    │
│                                                                                                                 │
│  Detailed Rationale:                                                                                            │
│  The customer, Jamie, has a low credit score of 325 and a closed account status, which places him in the        │
│  high-risk category. This is based on our risk level determination rules. Consequently, the applicable          │
│  interest rate for high-risk customers is 6.325%. Despite the high risk, Jamie is a PR holder. Our mandatory    │
│  compliance rule states that Non-Singaporeans without PR status must be rejected. However, since Jamie has PR   │
│  status, he is not subject to this rule. Therefore, my recommendation is to approve the loan, but at a higher   │
│  interest rate due to the high risk.                                                                            │
│                                                                                                                 │
╰────────────────────────────────────────────────────────

Output()

Would you like to view your execution traces? [y/N] (20s timeout): 